LMS Algorithm Implementation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
class LMSFilter:
    """Least Mean square adaptive filter"""
    def __init__(self, n_taps: int, learning_rate: float = 0.01):
        self.n_taps = n_taps
        self.learning_rate = learning_rate
        self.weights = np.zeros(n_taps)
        self.error_history = []
        self.weight_history = []

    def filter(self, x: np.ndarray, d: np.ndarray):
        """LMS Filter apply"""
        n_samples = len(x)
        y = np.zeros(n_samples)

        for n in range(self.n_taps, n_samples):
            x_n = x[n-self.n_taps:n][::-1] #Reverse for convolution

            y[n] = np.dot(self.weights, x_n)

            error = d[n] - y[n]
            self.error_history.append(error)

            self.weights += self.learning_rate * error * x_n
            self.weight_history.append(self.weights.copy())

        return y

    def plot_learning_curve(self):
        plt.figure(figsize=(14, 8))

        plt.subplot(2, 2, 1)
        plt.plot(np.abs(self.error_history))
        plt.title('Absolute Error vs Iterations')
        plt.xlabel('Iteration')
        plt.ylabel('|Error|')
        plt.yscale('log')
        plt.grid(True)

        plt.